In [37]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import time
from datetime import date, timedelta
import csv
import os

odds_file = r"D:\WORKSPACE\dotabet\data\odds\pinnacle.csv"
if not os.path.isfile(odds_file):
    with open(odds_file, mode='w', newline='') as file:
        writer = csv.writer(file)
        header = ["Match_Date", "League_Name", "Team_1", "Odd_1", "Team_2", "Odd_2"]
        writer.writerow(header)

today = date.today()
today_date = today.strftime('%Y-%m-%d')
tommorow_date = today + timedelta(days=1)

# Initialize WebDriver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

try:
    # Navigate to the webpage
    driver.get("https://www.pinnacle.com/en/esports/games/dota-2/matchups")

    # Allow some time for the page to load
    time.sleep(5)  # Adjust time based on your internet speed

    # Find all content blocks that might contain match details for Today or Tomorrow
    date_blocks = driver.find_elements(By.CSS_SELECTOR, "div.contentBlock.square > div.contentBlock.square")

    for date_block in date_blocks:
        # Extract the date (e.g., Today, Tomorrow)
        date = date_block.find_element(By.CSS_SELECTOR, "div.style_dateBar__1adEH").text.split(' ')[0]
        if date.split('\n')[0] == "TODAY":
            date_ymd = today_date
        elif date.split('\n')[0] == "TOMMOROW":
            date_ymd = tommorow_date
        else:
            date_ymd = None
        
        # Find all league blocks within the date block
        league_blocks = date_block.find_elements(By.CSS_SELECTOR, "div.style_row__yBzX8.style_row__3l5MS")

        for league_block in league_blocks:
            # Extract the league name
            league_name = league_block.find_element(By.CSS_SELECTOR, "div.style_metadata__3MrIC > a > span").text

            # Extract match rows within the league block
            match_blocks = league_block.find_elements(By.XPATH, "following-sibling::div")

            for match in match_blocks:
                # Stop if we hit another league block or end of date section
                if match.get_attribute("class") == "style_row__yBzX8 style_row__3l5MS":
                    break

                # Filter only rows with '(Match)' in the participant name
                participants = match.find_elements(By.CSS_SELECTOR, "div.style_gameInfoLabel__2m_fI > span")
                match_info = [p.text for p in participants if '(Match)' in p.text]

                if match_info:
                    # Extract Money Line odds
                    money_line_buttons = match.find_elements(By.CSS_SELECTOR, "div.style_moneyline__2xTld > div > button")
                    odds = [btn.find_element(By.CSS_SELECTOR, "span.style_price__3Haa9").text for btn in money_line_buttons]

                    
                    print(f"Date: {date_ymd}")
                    print(f"League: {league_name}")
                    print("Teams and Odds:")
                    data = [date_ymd, league_name]
                    for team, odd in zip(match_info, odds):
                        team = team.split(' (Match)')[0]
                        data.extend([team, odd])
                        print(f"{team}: {odd}")
                    print("----------")
                    with open(odds_file, mode='a', newline='') as file:
                        writer = csv.writer(file)
                        writer.writerow(data)
except Exception as e:
    print(f"An error occurred: {e}")
finally:
    # Clean up: close the browser
    driver.quit()


Date: 2024-05-03
League: EUROPEAN PRO LEAGUE
Teams and Odds:
Level UP MB: 6.870
Nemiga: 1.066
----------
Date: 2024-05-03
League: RES REGIONAL SERIES LATAM
Teams and Odds:
Estar Backs: 1.561
Infinity: 2.330
----------
Date: 2024-05-03
League: RES REGIONAL SERIES LATAM
Teams and Odds:
beastcoast: 1.400
Qhali: 2.810
----------


In [36]:
nik = 'Level UP MB (Match)'
nik = nik.split(' (Match)')[0]
nik

'Level UP MB'

In [1]:
import os
import requests
import requests
import subprocess
from datetime import datetime
from dotabet.script.pinnacle import scrape_dota2_odds

def send_telegram_message(token, chat_id, message):
    url = f"https://api.telegram.org/bot{token}/sendMessage"
    payload = {
        'chat_id': chat_id,
        'text': message
    }
    response = requests.post(url, data=payload)
    if response.status_code == 200:
        print("Message sent successfully.")
    else:
        print("Failed to send message.")

token = "6056809505:AAEci0XQUkG86E0PV1P0ujabDfYqkyUHtso"
chat_id = "333091706"
telegram_msg = scrape_dota2_odds()
send_telegram_message(token, chat_id, "\n\n".join(telegram_msg))
    

date_ymd='2024-05-03'
 league_name='EUROPEAN PRO LEAGUE'
 team1='Level UP MB' odd1='6.470' 
 team2='Nemiga' odd2='1.074'
----------

❗Data already exist in CSV
date_ymd='2024-05-03'
 league_name='RES REGIONAL SERIES LATAM'
 team1='Estar Backs' odd1='1.578' 
 team2='Infinity' odd2='2.300'
----------

❗Data already exist in CSV
date_ymd='2024-05-03'
 league_name='RES REGIONAL SERIES LATAM'
 team1='beastcoast' odd1='1.400' 
 team2='Qhali' odd2='2.810'
----------

❗Data already exist in CSV
Message sent successfully.
